In [ ]:

filename = "TEST"
piano = "Mason&Hamlin"

def set_filenames():
    global filename
    global piano  
    global originalMIDI
    global originalTXT
    global originalPNG
    global originalAUDIO
    global originalVIDEO
    global generatedMIDI
    global generatedTXT
    global generatedPNG
    global generatedAUDIO
    global generatedVIDEO
    originalMIDI = filename + ".mid"
    originalTXT = filename + ".txt"
    originalPNG = filename + ".png"
    originalAUDIO = filename + ".wav"
    originalVIDEO = filename + ".mp4"
    generatedMIDI = filename + "_out.mid"
    generatedTXT = filename + "_out.txt"
    generatedPNG = filename + "_out.png"
    generatedAUDIO = filename + "_out.wav"
    generatedVIDEO = filename + "_out.mp4"
set_filenames()
print("original MIDI filename:", originalMIDI)
print("original TXT filename:", originalTXT)
print("original PNG filename:", originalPNG)
print("original AUDIO filename:", originalAUDIO)
print("original VIDEO filename:", originalVIDEO)
print("generated MIDI filename:", generatedMIDI)
print("generated TXT filename:", generatedTXT)
print("generated PNG filename:", generatedPNG)
print("generated VIDEO filename:", generatedAUDIO)
from IPython.display import Audio

In [ ]:
from mido import Message, MidiFile, MidiTrack, MetaMessage

step = 14

# Create a new MIDI file and track
mid = MidiFile()  # default ticks_per_beat is 480
track = MidiTrack()
mid.tracks.append(track)

# (Optional) Set the tempo to 500000 microseconds per beat (120 BPM)
track.append(MetaMessage('set_tempo', tempo=500000))

# Define the range of notes from 1 to 88 (inclusive)
notes = list(range(21, 109))

# Total duration is 1 second = 960 ticks (2 beats at 480 ticks/beat)
# Divide equally among the notes so each note lasts for note_duration ticks.
# (Here, note_duration is kept as 120 ticks as in the original script.)
note_duration = 240

# Iterate over velocity values with a step of 'step'
#for velocity_value in range(119, 129, 1):
for velocity_value in range(2, 129, step):
    print("Velocity value: " + str(velocity_value - 1))
    for note in notes:
        print("Note number: " + str(note))
        # Note on event (delta time = 0 for immediate playing)
        track.append(Message('note_on', note=note, velocity=velocity_value - 1, time=0))
        # Note off event after 'note_duration' ticks
        track.append(Message('note_off', note=note, velocity=0, time=note_duration))

# Save the MIDI file
mid.save(originalMIDI)
print("Generated MIDI file: " + originalMIDI)

In [ ]:
import mido
import matplotlib.pyplot as plt

def midi_to_text_and_plot(midi_path, text_path, plot_path, title_text):
    """
    Decodes a MIDI file, writes its events into a text file, and plots:
    1) Velocity vs. Time (ignoring velocity 0)
    2) Histogram of Velocity Values with 128 bins (one per velocity level)
    Args:
        midi_path (str): Path to the input MIDI file.
        text_path (str): Path to save the output text file.
        plot_path (str): Path to save the generated plot as a PNG file.
        title_text (str): Text to be displayed in the upper left corner of the plot.
    """
    midi_file = mido.MidiFile(midi_path)
    ticks_per_beat = midi_file.ticks_per_beat
    tempo = 500000  # Default 120 BPM tempo (500,000 microseconds per beat)

    # Store extracted data
    velocity_values = []
    velocity_times = []

    with open(text_path, 'w') as text_file:
        text_file.write(f"MIDI File: {midi_path}\n")
        text_file.write(f"Ticks per Beat: {ticks_per_beat}\n\n")

        absolute_ticks = 0  # Tracks cumulative time in ticks

        for i, track in enumerate(midi_file.tracks):
            text_file.write(f"Track {i}: {track.name}\n")
            text_file.write("-" * 40 + "\n")

            for msg in track:
                absolute_ticks += msg.time  # Convert relative to absolute time

                # Tempo Change Handling (Optional)
                if msg.type == 'set_tempo':
                    tempo = msg.tempo  # Set new tempo

                # Convert MIDI ticks to seconds
                seconds = mido.tick2second(absolute_ticks, ticks_per_beat, tempo)

                # Note-On Event (Velocity > 0)
                if msg.type == 'note_on' and msg.velocity > 0:
                    velocity_values.append(msg.velocity)  # Store velocity
                    velocity_times.append(seconds)       # Store time

                    text_file.write(f"Time {seconds:.3f} sec | NOTE_ON  | "
                                    f"Note: {msg.note:3d} | Velocity: {msg.velocity:3d}\n")

                # Note-Off Event
                elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
                    text_file.write(f"Time {seconds:.3f} sec | NOTE_OFF | "
                                    f"Note: {msg.note:3d} | Velocity: 0\n")

            text_file.write("\n")


    # Generate Plots as Subplots
    if velocity_values:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))

        # Plot 1: Velocity vs. Time (Left subplot)
        axes[0].scatter(velocity_times, velocity_values, color='b', alpha=0.7, label="Note Velocity")
        axes[0].set_xlabel("Time (seconds)")
        axes[0].set_ylabel("Velocity (0-127)")
        axes[0].set_title("Velocity vs. Time")
        axes[0].legend()
        axes[0].grid()
        axes[0].text(-0.07, 1.05, title_text, transform=axes[0].transAxes, fontsize=12, verticalalignment='top')

        # Plot 2: Histogram of Velocity Values (Right subplot)
        axes[1].hist(velocity_values, bins=128, range=(0, 127), color='g', alpha=0.7, edgecolor='black')
        axes[1].set_xlabel("Velocity")
        axes[1].set_ylabel("Number of Occurrences")
        axes[1].set_title("Histogram of Velocity Values")
        axes[1].grid()

        # Adjust layout, save the plot, and display
        plt.tight_layout()
        plt.savefig(plot_path)  # Save plot as PNG file
        plt.show()

    print(f"PLOTS saved to: {plot_path}")
    print(f"MIDI events saved to: {text_path}")


midi_to_text_and_plot(originalMIDI, originalTXT, originalPNG, filename )



In [ ]:
import subprocess
import ipywidgets as widgets
from IPython.display import display, Audio
import os

# Global variable for the selected SoundFont name.
selected_sf_name = piano

# Define an output audio filename.
originalAUDIO = originalAUDIO

# Define available SoundFonts with your provided paths.
soundfonts = {
    "Mason&Hamlin": "/Users/user/.soundfonts/MasonHamlin-A-v7.sf2",
    "MuseScore": "/Users/user/.soundfonts/MuseScore_General.sf2",
    "4U_Steinway": "/Users/user/.soundfonts/4U-Steinway-v3.6.sf2",
    "Steinway_Chateau": "/Users/user/.soundfonts/Steinway-Chateau-Plus-Instruments-v1.7.sf2",
    "NY_S&S_B": "/Users/user/.soundfonts/Dore Mark's NY S&S Model B-v5.2.sf2",
    "Nice_Steinway": "/Users/user/.soundfonts/Nice-Steinway-v3.9.sf2",
    "Fazioli_v2.5": "/Users/user/.soundfonts/Dore Mark's (SF) Fazioli-v2.5.sf2",
    "Fazioli_F308": "/Users/user/.soundfonts/Dore Mark's Fazioli F308-v3.0.sf2",
    "Yamaha_C5": "/Users/user/.soundfonts/Yamaha C5 Grand-v2.4.sf2",
    "Yamaha_S6": "/Users/user/.soundfonts/Dore Mark's Yamaha S6-v1.6.sf2",
    "Clementi_Fortepiano1_808": "/Users/user/.soundfonts/Clementi Fortepiano 1808 (Dore Mark)-v1.0.sf2"
}

# Define available MIDI files (update these paths as needed)
midi_files = {
#    "Step14": "Step_14.mid",
    "original MIDI": originalMIDI
}

# Create radio buttons for SoundFont selection.
soundfont_selector = widgets.RadioButtons(
    options=list(soundfonts.keys()),
    description="SoundFont:",
    disabled=False
)

# Create radio buttons for MIDI file selection.
midi_selector = widgets.RadioButtons(
    options=list(midi_files.keys()),
    description="MIDI:",
    disabled=False
)

# Create a button widget to trigger synthesis.
synthesize_button = widgets.Button(
    description="Synthesize Audio",
    disabled=False,
    button_style='',  # Options: 'success', 'info', 'warning', 'danger'
    tooltip='Click to synthesize audio',
)

# Create an output widget to display messages.
output = widgets.Output()

def on_synthesize_button_clicked(b):
    global selected_sf_name  # Keep the variable global as requested.
    with output:
        output.clear_output()
        # Retrieve selected SoundFont and MIDI file paths.
        selected_sf_name = soundfont_selector.value
        selected_midi_name = midi_selector.value
        soundfont_path = soundfonts[selected_sf_name]
        midi_file_path = midi_files[selected_midi_name]
        
        # Verify that the files exist.
        if not os.path.exists(soundfont_path):
            print(f"SoundFont file not found: {soundfont_path}")
            return
        if not os.path.exists(midi_file_path):
            print(f"MIDI file not found: {midi_file_path}")
            return
   
        print("Selected SoundFont:", selected_sf_name)
        print("Selected MIDI:", selected_midi_name)
        print("Output audio file will be:", originalAUDIO)
        
        # Build the FluidSynth command.
        sample_rate = 44100
        command = [
            "fluidsynth",
            "-ni", 
            soundfont_path,
            midi_file_path,
            "-F", originalAUDIO,
            "-r", str(sample_rate)
        ]
        print("Running command:", " ".join(command))
        try:
            subprocess.run(command, check=True)
            print("Synthesis finished. Audio written to", originalAUDIO)
        except subprocess.CalledProcessError as e:
            print("Error during synthesis:", e)

# Bind the button click event to the callback function.
synthesize_button.on_click(on_synthesize_button_clicked)

# Display the widgets.
display(soundfont_selector, midi_selector, synthesize_button, output)

In [ ]:
Audio(originalAUDIO)

In [ ]:
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"  # Allow Pygame to run in headless mode
import mido
import pygame
import moviepy.editor as mpy
#from moviepy.config import change_settings
from pygame.locals import *

import moviepy.config as mpy_config
import imageio_ffmpeg
# Use the ffmpeg binary provided by imageio_ffmpeg.
mpy_config.change_settings({"FFMPEG_BINARY": imageio_ffmpeg.get_ffmpeg_exe()})

# The following line has been removed because it forces MoviePy to use an ffmpeg binary that does not exist.
# change_settings({"FFMPEG_BINARY": "/opt/homebrew/bin/ffmpeg"})

# Constants for visualization
WHITE_KEY_WIDTH = 20
WHITE_KEY_HEIGHT = 120
BLACK_KEY_WIDTH = int(WHITE_KEY_WIDTH * 0.65)  # Black keys are 65% the width of white keys
BLACK_KEY_HEIGHT = int(WHITE_KEY_HEIGHT * 0.7)  # Black keys are 70% the height of white keys
FPS = 30
VIDEO_OUTPUT = "keyboard_visualization_with_audio.mp4"

# Map MIDI notes to key positions
# (Each tuple represents a key: the letter indicates whether the key is white ("W") or black ("B")
#  and the number is its relative position in the octave.)
KEYS = [
    ('W', 0), ('B', 1), ('W', 2), ('W', 3), ('B', 4),
    ('W', 5), ('B', 6), ('W', 7), ('W', 8), ('B', 9), ('W', 10), ('B', 11)
]
NUM_WHITE_KEYS = 52
NUM_BLACK_KEYS = 36
NUM_KEYS = 88
START_NOTE = 21  # MIDI number for A0

def note_to_position(note):
    """Converts a MIDI note number to its position on the keyboard."""
    key_offset = note - START_NOTE
    if key_offset < 0 or key_offset >= NUM_KEYS:
        return None, None  # Ignore notes outside the keyboard range

    octave = key_offset // 12
    key_index = key_offset % 12
    key_type, _ = KEYS[key_index]

    if key_type == 'W':
        white_index = sum(1 for k, _ in KEYS[:key_index] if k == 'W') + octave * 7
        return white_index, 'W'
    elif key_type == 'B':
        black_index = sum(1 for k, _ in KEYS[:key_index] if k == 'B') + octave * 5
        return black_index, 'B'

    return None, None

def parse_midi(midi_file):
    """Parses MIDI events and returns note on/off events with timestamps."""
    midi = mido.MidiFile(midi_file)
    events = []
    time = 0
    for msg in midi:
        time += msg.time
        # Only process note_on and note_off events.
        if msg.type in ['note_on', 'note_off']:
            # For note_on, use the velocity; for note_off, use 0.
            events.append((time, msg.note, msg.velocity if msg.type == 'note_on' else 0))
        # Any pedal events (control_change with control 64) are ignored.
    return events

def visualize_keyboard(events, output_path, audio_file):
    """Generates a video visualizing the keyboard based on MIDI events and includes audio."""
    # Initialize pygame in headless mode
    pygame.init()
    screen_width = WHITE_KEY_WIDTH * NUM_WHITE_KEYS
    screen_height = WHITE_KEY_HEIGHT + 20  # Add extra space for border
    screen = pygame.Surface((screen_width, screen_height))

    # Prepare the list of frames.
    frames = []
    white_keys_pressed = [False] * NUM_WHITE_KEYS
    black_keys_pressed = [False] * NUM_BLACK_KEYS
    event_index = 0

    def draw_keyboard():
        """Draws the keyboard (with any keys highlighted that are currently 'pressed')."""
        # Draw a border (frame) around the keyboard.
        pygame.draw.rect(screen, (50, 50, 50),
                         (0, 0, WHITE_KEY_WIDTH * NUM_WHITE_KEYS, WHITE_KEY_HEIGHT + 10))

        # Draw white keys.
        for i in range(NUM_WHITE_KEYS):
            color = (255, 255, 255) if not white_keys_pressed[i] else (200, 0, 0)
            pygame.draw.rect(screen, color, (i * WHITE_KEY_WIDTH, 10, WHITE_KEY_WIDTH - 1, WHITE_KEY_HEIGHT))
            pygame.draw.rect(screen, (0, 0, 0), (i * WHITE_KEY_WIDTH, 10, WHITE_KEY_WIDTH - 1, WHITE_KEY_HEIGHT), 1)

        # Draw black keys.
        black_key_offsets = [0.6, 2.6, 3.6, 5.6, 6.6]  # Offsets for black keys within an octave.
        for octave in range(7):  # For 7 octaves from A0 to B7 (roughly).
            for i, offset in enumerate(black_key_offsets):
                black_index = octave * 5 + i
                if 0 <= black_index < NUM_BLACK_KEYS:
                    x_pos = (octave * 7 + offset) * WHITE_KEY_WIDTH
                    color = (0, 0, 0) if not black_keys_pressed[black_index] else (200, 0, 0)
                    pygame.draw.rect(screen, color, (x_pos, 10, BLACK_KEY_WIDTH, BLACK_KEY_HEIGHT))

        # Manually draw the last black key (B7) if needed.
        last_black_key_x = (49 * WHITE_KEY_WIDTH) + (0.6 * WHITE_KEY_WIDTH)
        color = (0, 0, 0) if not black_keys_pressed[-1] else (200, 0, 0)
        pygame.draw.rect(screen, color, (last_black_key_x, 10, BLACK_KEY_WIDTH, BLACK_KEY_HEIGHT))

    max_time = events[-1][0] if events else 0
    # Process frames at the designated FPS.
    for frame_time in range(int(max_time * FPS)):
        current_time = frame_time / FPS
        # Process all events up to the current time.
        while event_index < len(events) and events[event_index][0] <= current_time:
            _, note, velocity = events[event_index]
            key_index, key_type = note_to_position(note)
            if key_index is not None:
                if key_type == 'W':
                    white_keys_pressed[key_index] = (velocity > 0)
                elif key_type == 'B':
                    black_keys_pressed[key_index] = (velocity > 0)
            event_index += 1

        draw_keyboard()
        # Convert the Pygame surface into an image frame (RGB).
        frames.append(pygame.surfarray.array3d(screen).transpose((1, 0, 2)))

    # Create the video using MoviePy.
    def make_frame(t):
        frame_index = min(int(t * FPS), len(frames) - 1)
        return frames[frame_index]

    print("Loading audio clip...")
    audio = mpy.AudioFileClip(audio_file)
    
    print("Building video clip...")
    clip = mpy.VideoClip(make_frame, duration=max_time)
    clip = clip.set_audio(audio)

    print("Writing final video...")
    clip.write_videofile(
        output_path,
        fps=FPS,
        codec="libx264",
        audio_codec="aac",
        ffmpeg_params=["-pix_fmt", "yuv420p"]
    )
    print(f"Video saved to: {output_path}")

# ----------------- EXECUTION BLOCK ----------------- #
# Adjust these file paths as needed
midi_file = originalMIDI
audio_file = originalAUDIO
output_video = originalVIDEO

# Check if files exist
if not os.path.exists(midi_file):
    print(f"MIDI file not found: {midi_file}")
elif not os.path.exists(audio_file):
    print(f"Audio file not found: {audio_file}")
else:
    print("Parsing MIDI file...")
    midi_events = parse_midi(midi_file)
    print("Generating video...")
    visualize_keyboard(midi_events, output_video, audio_file)


In [ ]:
from IPython.display import Video
Video(originalVIDEO, embed=True)

In [ ]:
# transcription from AUDIO (wav) to MIDI (mid)
!python example.py --audio_path={originalAUDIO} --output_midi_path={generatedMIDI}


In [ ]:
import mido
import matplotlib.pyplot as plt

def midi_to_text_and_plot(midi_path, text_path, plot_path, title_text):
    """
    Decodes a MIDI file, writes its events into a text file, and plots:
    1) Velocity vs. Time (ignoring velocity 0)
    2) Histogram of Velocity Values with 128 bins (one per velocity level)

    Args:
        midi_path (str): Path to the input MIDI file.
        text_path (str): Path to save the output text file.
        plot_path (str): Path to save the generated plot as a PNG file.
        title_text (str): Text to be displayed in the upper left corner of the plot.
    """
    midi_file = mido.MidiFile(midi_path)
    ticks_per_beat = midi_file.ticks_per_beat
    tempo = 500000  # Default 120 BPM tempo (500,000 microseconds per beat)

    # Store extracted data
    velocity_values = []
    velocity_times = []

    with open(text_path, 'w') as text_file:
        text_file.write(f"MIDI File: {midi_path}\n")
        text_file.write(f"Ticks per Beat: {ticks_per_beat}\n\n")

        absolute_ticks = 0  # Tracks cumulative time in ticks

        for i, track in enumerate(midi_file.tracks):
            text_file.write(f"Track {i}: {track.name}\n")
            text_file.write("-" * 40 + "\n")

            for msg in track:
                absolute_ticks += msg.time  # Convert relative to absolute time

                # Tempo Change Handling (Optional)
                if msg.type == 'set_tempo':
                    tempo = msg.tempo  # Set new tempo

                # Convert MIDI ticks to seconds
                seconds = mido.tick2second(absolute_ticks, ticks_per_beat, tempo)

                # Note-On Event (Velocity > 0)
                if msg.type == 'note_on' and msg.velocity > 0:
                    velocity_values.append(msg.velocity)  # Store velocity
                    velocity_times.append(seconds)       # Store time

                    text_file.write(f"Time {seconds:.3f} sec | NOTE_ON  | "
                                    f"Note: {msg.note:3d} | Velocity: {msg.velocity:3d}\n")

                # Note-Off Event
                elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
                    text_file.write(f"Time {seconds:.3f} sec | NOTE_OFF | "
                                    f"Note: {msg.note:3d} | Velocity: 0\n")

            text_file.write("\n")

    print(f"MIDI events saved to: {text_path}")

    # Generate Plots as Subplots
    if velocity_values:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))

        # Plot 1: Velocity vs. Time (Left subplot)
        axes[0].scatter(velocity_times, velocity_values, color='b', alpha=0.7, label="Note Velocity")
        axes[0].set_xlabel("Time (seconds)")
        axes[0].set_ylabel("Velocity (0-127)")
        axes[0].set_title("Velocity vs. Time")
        axes[0].legend()
        axes[0].grid()
        axes[0].text(-0.07, 1.05, title_text, transform=axes[0].transAxes, fontsize=12, verticalalignment='top')

        # Plot 2: Histogram of Velocity Values (Right subplot)
        axes[1].hist(velocity_values, bins=128, range=(0, 127), color='g', alpha=0.7, edgecolor='black')
        axes[1].set_xlabel("Velocity")
        axes[1].set_ylabel("Number of Occurrences")
        axes[1].set_title("Histogram of Velocity Values")
        axes[1].grid()

        # Adjust layout, save the plot, and display
        plt.tight_layout()
        plt.savefig(plot_path)  # Save plot as PNG file
        plt.show()

    print(f"PLOTS saved to: {plot_path}")
    print(f"MIDI events saved to: {text_path}")

midi_to_text_and_plot(generatedMIDI, filename  + selected_sf_name + "_out.txt", filename + selected_sf_name + "_out.png", selected_sf_name)


In [ ]:
import subprocess
import ipywidgets as widgets
from IPython.display import display, Audio
import os

# Global variable for the selected SoundFont name.
selected_sf_name = piano

# Define an output audio filename.
generatedAUDIO = generatedAUDIO

# Define available SoundFonts with your provided paths.
soundfonts = {
    "Mason&Hamlin": "/Users/user/.soundfonts/MasonHamlin-A-v7.sf2",
    "MuseScore": "/Users/user/.soundfonts/MuseScore_General.sf2",
    "4U_Steinway": "/Users/user/.soundfonts/4U-Steinway-v3.6.sf2",
    "Steinway_Chateau": "/Users/user/.soundfonts/Steinway-Chateau-Plus-Instruments-v1.7.sf2",
    "NY_S&S_B": "/Users/user/.soundfonts/Dore Mark's NY S&S Model B-v5.2.sf2",
    "Nice_Steinway": "/Users/user/.soundfonts/Nice-Steinway-v3.9.sf2",
    "Fazioli_v2.5": "/Users/user/.soundfonts/Dore Mark's (SF) Fazioli-v2.5.sf2",
    "Fazioli_F308": "/Users/user/.soundfonts/Dore Mark's Fazioli F308-v3.0.sf2",
    "Yamaha_C5": "/Users/user/.soundfonts/Yamaha C5 Grand-v2.4.sf2",
    "Yamaha_S6": "/Users/user/.soundfonts/Dore Mark's Yamaha S6-v1.6.sf2",
    "Clementi_Fortepiano1_808": "/Users/user/.soundfonts/Clementi Fortepiano 1808 (Dore Mark)-v1.0.sf2"
}

# Define available MIDI files (update these paths as needed)
midi_files = {
#    "Step14": "Step_14.mid",
    "generated MIDI": generatedMIDI
}

# Create radio buttons for SoundFont selection.
soundfont_selector = widgets.RadioButtons(
    options=list(soundfonts.keys()),
    description="SoundFont:",
    disabled=False
)

# Create radio buttons for MIDI file selection.
midi_selector = widgets.RadioButtons(
    options=list(midi_files.keys()),
    description="MIDI:",
    disabled=False
)

# Create a button widget to trigger synthesis.
synthesize_button = widgets.Button(
    description="Synthesize Audio",
    disabled=False,
    button_style='',  # Options: 'success', 'info', 'warning', 'danger'
    tooltip='Click to synthesize audio',
)

# Create an output widget to display messages.
output = widgets.Output()

def on_synthesize_button_clicked(b):
    global selected_sf_name  # Keep the variable global as requested.
    with output:
        output.clear_output()
        # Retrieve selected SoundFont and MIDI file paths.
        selected_sf_name = soundfont_selector.value
        selected_midi_name = midi_selector.value
        soundfont_path = soundfonts[selected_sf_name]
        midi_file_path = midi_files[selected_midi_name]
        
        # Verify that the files exist.
        if not os.path.exists(soundfont_path):
            print(f"SoundFont file not found: {soundfont_path}")
            return
        if not os.path.exists(midi_file_path):
            print(f"MIDI file not found: {midi_file_path}")
            return
   
        print("Selected SoundFont:", selected_sf_name)
        print("Selected MIDI:", selected_midi_name)
        print("Output audio file will be:", generatedAUDIO)
        
        # Build the FluidSynth command.
        sample_rate = 44100
        command = [
            "fluidsynth",
            "-ni", 
            soundfont_path,
            midi_file_path,
            "-F", generatedAUDIO,
            "-r", str(sample_rate)
        ]
        print("Running command:", " ".join(command))
        try:
            subprocess.run(command, check=True)
            print("Synthesis finished. Audio written to", generatedAUDIO)
        except subprocess.CalledProcessError as e:
            print("Error during synthesis:", e)

# Bind the button click event to the callback function.
synthesize_button.on_click(on_synthesize_button_clicked)

# Display the widgets.
display(soundfont_selector, midi_selector, synthesize_button, output)

In [ ]:
Audio(generatedAUDIO)

In [ ]:
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"  # Allow Pygame to run in headless mode
import mido
#import pygame
import moviepy.editor as mpy
#from moviepy.config import change_settings
from pygame.locals import *

import moviepy.config as mpy_config
import imageio_ffmpeg
# Use the ffmpeg binary provided by imageio_ffmpeg.
mpy_config.change_settings({"FFMPEG_BINARY": imageio_ffmpeg.get_ffmpeg_exe()})

# The following line has been removed because it forces MoviePy to use an ffmpeg binary that does not exist.
# change_settings({"FFMPEG_BINARY": "/opt/homebrew/bin/ffmpeg"})

# Constants for visualization
WHITE_KEY_WIDTH = 20
WHITE_KEY_HEIGHT = 120
BLACK_KEY_WIDTH = int(WHITE_KEY_WIDTH * 0.65)  # Black keys are 65% the width of white keys
BLACK_KEY_HEIGHT = int(WHITE_KEY_HEIGHT * 0.7)  # Black keys are 70% the height of white keys
FPS = 30
VIDEO_OUTPUT = "keyboard_visualization_with_audio.mp4"

# Map MIDI notes to key positions
# (Each tuple represents a key: the letter indicates whether the key is white ("W") or black ("B")
#  and the number is its relative position in the octave.)
KEYS = [
    ('W', 0), ('B', 1), ('W', 2), ('W', 3), ('B', 4),
    ('W', 5), ('B', 6), ('W', 7), ('W', 8), ('B', 9), ('W', 10), ('B', 11)
]
NUM_WHITE_KEYS = 52
NUM_BLACK_KEYS = 36
NUM_KEYS = 88
START_NOTE = 21  # MIDI number for A0

def note_to_position(note):
    """Converts a MIDI note number to its position on the keyboard."""
    key_offset = note - START_NOTE
    if key_offset < 0 or key_offset >= NUM_KEYS:
        return None, None  # Ignore notes outside the keyboard range

    octave = key_offset // 12
    key_index = key_offset % 12
    key_type, _ = KEYS[key_index]

    if key_type == 'W':
        white_index = sum(1 for k, _ in KEYS[:key_index] if k == 'W') + octave * 7
        return white_index, 'W'
    elif key_type == 'B':
        black_index = sum(1 for k, _ in KEYS[:key_index] if k == 'B') + octave * 5
        return black_index, 'B'

    return None, None

def parse_midi(midi_file):
    """Parses MIDI events and returns note on/off events with timestamps."""
    midi = mido.MidiFile(midi_file)
    events = []
    time = 0
    for msg in midi:
        time += msg.time
        # Only process note_on and note_off events.
        if msg.type in ['note_on', 'note_off']:
            # For note_on, use the velocity; for note_off, use 0.
            events.append((time, msg.note, msg.velocity if msg.type == 'note_on' else 0))
        # Any pedal events (control_change with control 64) are ignored.
    return events

def visualize_keyboard(events, output_path, audio_file):
    """Generates a video visualizing the keyboard based on MIDI events and includes audio."""
    # Initialize pygame in headless mode
    pygame.init()
    screen_width = WHITE_KEY_WIDTH * NUM_WHITE_KEYS
    screen_height = WHITE_KEY_HEIGHT + 20  # Add extra space for border
    screen = pygame.Surface((screen_width, screen_height))

    # Prepare the list of frames.
    frames = []
    white_keys_pressed = [False] * NUM_WHITE_KEYS
    black_keys_pressed = [False] * NUM_BLACK_KEYS
    event_index = 0

    def draw_keyboard():
        """Draws the keyboard (with any keys highlighted that are currently 'pressed')."""
        # Draw a border (frame) around the keyboard.
        pygame.draw.rect(screen, (50, 50, 50),
                         (0, 0, WHITE_KEY_WIDTH * NUM_WHITE_KEYS, WHITE_KEY_HEIGHT + 10))

        # Draw white keys.
        for i in range(NUM_WHITE_KEYS):
            color = (255, 255, 255) if not white_keys_pressed[i] else (200, 0, 0)
            pygame.draw.rect(screen, color, (i * WHITE_KEY_WIDTH, 10, WHITE_KEY_WIDTH - 1, WHITE_KEY_HEIGHT))
            pygame.draw.rect(screen, (0, 0, 0), (i * WHITE_KEY_WIDTH, 10, WHITE_KEY_WIDTH - 1, WHITE_KEY_HEIGHT), 1)

        # Draw black keys.
        black_key_offsets = [0.6, 2.6, 3.6, 5.6, 6.6]  # Offsets for black keys within an octave.
        for octave in range(7):  # For 7 octaves from A0 to B7 (roughly).
            for i, offset in enumerate(black_key_offsets):
                black_index = octave * 5 + i
                if 0 <= black_index < NUM_BLACK_KEYS:
                    x_pos = (octave * 7 + offset) * WHITE_KEY_WIDTH
                    color = (0, 0, 0) if not black_keys_pressed[black_index] else (200, 0, 0)
                    pygame.draw.rect(screen, color, (x_pos, 10, BLACK_KEY_WIDTH, BLACK_KEY_HEIGHT))

        # Manually draw the last black key (B7) if needed.
        last_black_key_x = (49 * WHITE_KEY_WIDTH) + (0.6 * WHITE_KEY_WIDTH)
        color = (0, 0, 0) if not black_keys_pressed[-1] else (200, 0, 0)
        pygame.draw.rect(screen, color, (last_black_key_x, 10, BLACK_KEY_WIDTH, BLACK_KEY_HEIGHT))

    max_time = events[-1][0] if events else 0
    # Process frames at the designated FPS.
    for frame_time in range(int(max_time * FPS)):
        current_time = frame_time / FPS
        # Process all events up to the current time.
        while event_index < len(events) and events[event_index][0] <= current_time:
            _, note, velocity = events[event_index]
            key_index, key_type = note_to_position(note)
            if key_index is not None:
                if key_type == 'W':
                    white_keys_pressed[key_index] = (velocity > 0)
                elif key_type == 'B':
                    black_keys_pressed[key_index] = (velocity > 0)
            event_index += 1

        draw_keyboard()
        # Convert the Pygame surface into an image frame (RGB).
        frames.append(pygame.surfarray.array3d(screen).transpose((1, 0, 2)))

    # Create the video using MoviePy.
    def make_frame(t):
        frame_index = min(int(t * FPS), len(frames) - 1)
        return frames[frame_index]

    print("Loading audio clip...")
    audio = mpy.AudioFileClip(audio_file)
    
    print("Building video clip...")
    clip = mpy.VideoClip(make_frame, duration=max_time)
    clip = clip.set_audio(audio)

    print("Writing final video...")
    clip.write_videofile(
        output_path,
        fps=FPS,
        codec="libx264",
        audio_codec="aac",
        ffmpeg_params=["-pix_fmt", "yuv420p"]
    )
    print(f"Video saved to: {output_path}")

# ----------------- EXECUTION BLOCK ----------------- #
# Adjust these file paths as needed
midi_file = generatedMIDI
audio_file = generatedAUDIO
output_video = generatedVIDEO

# Check if files exist
if not os.path.exists(midi_file):
    print(f"MIDI file not found: {midi_file}")
elif not os.path.exists(audio_file):
    print(f"Audio file not found: {audio_file}")
else:
    print("Parsing MIDI file...")
    midi_events = parse_midi(midi_file)
    print("Generating video...")
    visualize_keyboard(midi_events, output_video, audio_file)


In [ ]:
from IPython.display import Video
Video(generatedVIDEO, embed=True)